Imports below:

In [ ]:
# Imports
import os
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Step 1 & 2: Load and Combine Data

In [ ]:
# Define the paths to your data files
fake_news_path = os.path.join("content", "Fake.csv")
true_news_path = os.path.join("content", "True.csv")

In [ ]:
# Load the data into pandas DataFrames
fake_df = pd.read_csv(fake_news_path)
true_df = pd.read_csv(true_news_path)